# Enough with the `loss.backward()` already:<br/>DRY out your PyTorch code and gain superpowers with PyTorch Lightning

PyTorch is famous for its "what you see is what you get" coding style
* Define-by-run + implicit gradient calculation
__BUT__
* Explicit loss and backprop calculation
* Explicit weight updates
* Explicit training flow control
* Explicit device targeting

Let's review a quick example (data from https://archive.ics.uci.edu/ml/datasets/combined+cycle+power+plant)

In [ ]:
import pandas as pd

df = pd.read_csv('powerplant.csv')

df

We want to model PE (power output) as a function of the other variables. We'll use a simple MLP.

In [ ]:
import torch
from torch.utils.data import TensorDataset, DataLoader

X_train_pyt = torch.tensor(df[['AT', 'V', 'RH', 'AP']].values, dtype=torch.float)
y_train_pyt = torch.tensor(df.PE.values, dtype=torch.float)

size = len(df)
val_size = size // 10

X_train_pyt_main = X_train_pyt[:(size-val_size)]
X_train_pyt_val = X_train_pyt[(size-val_size):]

y_train_pyt_main = y_train_pyt[:(size-val_size)]
y_train_pyt_val = y_train_pyt[(size-val_size):]

train_ds = TensorDataset(X_train_pyt_main, y_train_pyt_main[:, None])
print(len(train_ds))

val_ds = TensorDataset(X_train_pyt_val, y_train_pyt_val[:,None])
print(len(val_ds))

def train(training, validation, epochs, batch_size, model, loss_fn, optimizer):
    history = { 'train' : [], 'val' : [] }
    for epoch in range(epochs):
        batch_losses = []
        for i in range((len(training) - 1) // batch_size + 1):
            xb, yb = training[i * batch_size: i * batch_size + batch_size]      
            pred = model(xb)
            loss = loss_fn(pred, yb)
            optimizer.zero_grad()
            loss.backward()        
            optimizer.step()
            batch_losses.append(loss.item())
        epoch_loss = pd.Series(batch_losses).mean()
        history['train'].append(epoch_loss)
        y_val_pred = model(validation.tensors[0])
        epoch_val = loss_fn(y_val_pred, validation.tensors[1]).item()
        history['val'].append(epoch_val) 
        print("MSE for epoch {} = Train {}, Val {}".format(epoch, epoch_loss, epoch_val))     
    return history

D_in, H1, D_out = 4, 8, 1

model = torch.nn.Sequential(
  torch.nn.Linear(D_in, H1), 
  torch.nn.ReLU(),
  torch.nn.Linear(H1, D_out)
)
optimizer = torch.optim.Adam(model.parameters())

In [ ]:
hist = train(train_ds, val_ds, 10, 50, model, torch.nn.MSELoss(), optimizer)

__This is a lot of boilerplate ... and this example (since it's meant to run in binder) doesn't even get into device targeting__

For many common use cases, 

1. We don't actually need (or want) that level of control/visibility
2. More code == more long-term maintainability cost

## Goal: DRY and Automate: Reducing/Removing Boilerplate, add Functionality

Now we want to simplify/automate that as well as provide more functionality like:
* Drive visual reporting tools like Tensorboard
* Integrate hooks for checkpointing, early stopping, etc.
* Prepare for distributed training and/or deploying to alternative hardware

In the past, there have been several tools that help with this...
* skorch https://github.com/skorch-dev/skorch
* Torchbearer https://github.com/pytorchbearer/torchbearer
* Ignite https://pytorch.org/ignite/index.html
* and a bunch of smaller projects

## Introducing PyTorch Lightning

Today, __PyTorch Lightning__ (https://www.pytorchlightning.ai/) has emerged as the dominant best-practices extension of PyTorch to meet our training and refactor needs.

> The ultimate PyTorch research framework. Scale your models, without the boilerplate.

Let's see it in action

In [ ]:
import pytorch_lightning as pl

## Where do we start with PyTorch Lightning?

Since we already have it installed and verified that it can load, the next step is

### Create a Lightning Module

Specifically:

1. Define a class that inherits from `pl.LightningModule` (a Lightning Module is actually just a flavor or regular PyTorch module!)
2. In the constructor, put our neural-net-building code (assign the model to an instance variable)
3. Add a `forward` method for the forward pass (return the output of model)
4. Add a `training_step` method (return the loss)
5. Add a `configure_optimizers` method (return your optimizer instance)
    
__We'll do this part one step at a time__

In [ ]:
class LitPowerplant(pl.LightningModule):

    def __init__(self):
        super().__init__()
        D_in, H1, D_out = 4, 8, 1

        self.model = torch.nn.Sequential(
            torch.nn.Linear(D_in, H1), 
            torch.nn.ReLU(),
            torch.nn.Linear(H1, D_out)
        )

Add `forward`

In [ ]:
class LitPowerplant(pl.LightningModule):

    def __init__(self):
        super().__init__()
        D_in, H1, D_out = 4, 8, 1

        self.model = torch.nn.Sequential(
            torch.nn.Linear(D_in, H1), 
            torch.nn.ReLU(),
            torch.nn.Linear(H1, D_out)
        )
    
    def forward(self, x):
        return self.model(x)

Next, `training_step`

In [ ]:
class LitPowerplant(pl.LightningModule):

    def __init__(self):
        super().__init__()
        D_in, H1, D_out = 4, 8, 1

        self.model = torch.nn.Sequential(
            torch.nn.Linear(D_in, H1), 
            torch.nn.ReLU(),
            torch.nn.Linear(H1, D_out)
        )
    
    def forward(self, x):
        return self.model(x)
    
    def training_step(self, batch, batch_idx):
        x, y = batch
        pred = self.model(x)
        loss_fn = torch.nn.MSELoss()
        loss = loss_fn(pred, y)
        self.log('train_loss', loss) #TensorBoard by default
        return loss

And then `configure_optimizers`

In [ ]:
class LitPowerplant(pl.LightningModule):

    def __init__(self):
        super().__init__()
        D_in, H1, D_out = 4, 8, 1

        self.model = torch.nn.Sequential(
            torch.nn.Linear(D_in, H1), 
            torch.nn.ReLU(),
            torch.nn.Linear(H1, D_out)
        )
    
    def forward(self, x):
        return self.model(x)
    
    def training_step(self, batch, batch_idx):
        x, y = batch
        pred = self.model(x)
        loss_fn = torch.nn.MSELoss()
        loss = loss_fn(pred, y)
        self.log('train_loss', loss) #TensorBoard by default
        return loss
    
    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.model.parameters())
        return optimizer
    
    # We can also add our validation and/or test logic (later we'd DRY this code by factoring out the common codes from training)
    def validation_step(self, batch, batch_idx):
        x, y = batch
        pred = self.model(x)
        loss_fn = torch.nn.MSELoss()
        loss = loss_fn(pred, y)
        self.log('val_loss', loss)

### Prepare our Data via DataLoader

This is a PyTorch pattern/best practice that we would often want or need to do anyway!

DataLoader represents a Python iterable over a dataset, with support for
* map-style and iterable-style datasets,
* customizing data loading order,
* automatic batching,
* single- and multi-process data loading,
* automatic memory pinning.

In [ ]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(train_ds, batch_size=64, shuffle=True)
validate_dataloader = DataLoader(val_ds, batch_size=64)

### The Big Payoff: Lighting Training

We are now ready to train with the Lightning `Trainer` object.

Let's get it running and then review the benefits.

In [ ]:
powerplant = LitPowerplant()

# trainer = pl.Trainer(gpus=8) (if you have GPUs)
trainer = pl.Trainer(max_epochs=12)
trainer.fit(powerplant, train_dataloader, validate_dataloader)

### Ok, what have we got?

__Under the hood, the Lightning Trainer handles the training loop details for you, some examples include:__
* Epoch and batch iteration (running the training, validation and test dataloaders)
* Calling of optimizer.step(), backward, zero_grad()
* Calling of .eval(), enabling/disabling grads
* Tensorboard (see loggers options)
* GPU/Multi-GPU support, TPU, AMP support
  * Putting batches and computations on the correct devices
* Calling the Callbacks at the appropriate times (e.g., https://pytorch-lightning.readthedocs.io/en/latest/common/trainer.html#checkpoint-callback)

### Viewing Tensorboard

If you're running locally (and you have TF installed), you can launch

`tensorboard --logdir ./lightning_logs`

And in Google Colab, we can use a notebook extension to render Tensorboard inline (for more info see https://colab.research.google.com/github/tensorflow/tensorboard/blob/master/docs/tensorboard_in_notebooks.ipynb)

In [ ]:
%load_ext tensorboard

%tensorboard --logdir ./lightning_logs

### One Last Bonus

Simple distributed training with Ray (https://www.ray.io/) and Ray Lightning (https://github.com/ray-project/ray_lightning)

```python
! pip install ray_lightning
from ray_lightning import RayPlugin

plugin = RayPlugin(num_workers=100, use_gpu=False, num_cpus_per_worker=1)

# Don't set `gpus` -- the actual number of GPUs is determined by `num_workers`

ray_trainer = pl.Trainer(max_epochs=100, plugins=[plugin])
ray_trainer.fit(powerplant, train_dataloader, validate_dataloader)
```